# Tunability for KNeighborsClassifier

# Intro

In [1]:
import pandas as pd

random_state = pd.read_csv("../../data/vars.csv")['random_state'].iloc[0]
n_iter = pd.read_csv("../../data/vars.csv")['n_iter'].iloc[0]
n_iter_BS = pd.read_csv("../../data/vars.csv")['n_iter_BS'].iloc[0]

In [2]:
df_college = pd.read_csv("../../data/processed/college.csv")
df_credit = pd.read_csv("../../data/processed/credit.csv")
df_diabetes = pd.read_csv("../../data/processed/diabetes.csv")
df_penguins = pd.read_csv("../../data/processed/penguins.csv")

In [3]:
print(df_college.info())
print()
print(df_credit.info())
print()
print(df_diabetes.info())
print()
print(df_penguins.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   bool   
 10  will_go_to_college     1000 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 79.2+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
import numpy as np


num_pipeline = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('cat_impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])

col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])

model_pipeline = Pipeline(steps=[
    ('preprocessing', col_trans),
    ('model', KNeighborsClassifier())
])

# RandomizedSearchCV

In [5]:
knn_params = {
    'model__n_neighbors': range(1, 31), # paper
    #'model__weights': ['uniform', 'distance'],
    #'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'model__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
}

## Credit dataset

In [6]:
from sklearn.model_selection import RandomizedSearchCV

X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]

random_search_model = RandomizedSearchCV(estimator = model_pipeline,
                                   param_distributions = knn_params,
                                   n_iter = n_iter,
                                   cv = 5,
                                   scoring = 'f1',
                                   random_state = random_state)

random_search_model.fit(X, y)

best_f1_credit = random_search_model.best_score_

print(f"Best f1 score: {best_f1_credit}")
print(f"Best params: {random_search_model.best_params_}")

random_search_model.best_estimator_

c:\Users\natal\.conda\envs\tunability\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 120 is smaller than n_iter=200. Running 120 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best f1 score: 0.8453157699330955
Best params: {'model__n_neighbors': 7, 'model__metric': 'manhattan'}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237D0482E90>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237D0483D90>)])),
                ('model',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=7))])

## Data frame with params, results and mean result

In [7]:
tested_params = random_search_model.cv_results_['params']

In [8]:
df_knn = pd.DataFrame(tested_params)
df_knn['credit_f1'] = random_search_model.cv_results_['mean_test_score']
df_knn

,model__n_neighbors,model__metric,credit_f1
0,1,euclidean,0.780474
1,2,euclidean,0.744687
2,3,euclidean,0.839961
3,4,euclidean,0.810188
4,5,euclidean,0.829706
...,...,...,...
115,26,minkowski,0.835261
116,27,minkowski,0.841074
117,28,minkowski,0.826965
118,29,minkowski,0.829547


## College dataset

In [9]:
from sklearn.model_selection import GridSearchCV

X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_knn['college_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_college = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_college}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.8451496931440531
Best params: {'model__metric': 'euclidean', 'model__n_neighbors': 3}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237D274E210>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237D274D210>)])),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=3))])

## Diabetes dataset

In [10]:
from sklearn.model_selection import GridSearchCV

X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_knn['diabetes_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_diabetes = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_diabetes}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.6441857768953202
Best params: {'model__metric': 'euclidean', 'model__n_neighbors': 11}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237D273E410>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237D26927D0>)])),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=11))])

## Penguins dataset

In [11]:
from sklearn.model_selection import GridSearchCV

X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_knn['penguins_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_penguins = grid_search_model.best_score_


print(f"Best f1 score: {best_f1_penguins}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 1.0
Best params: {'model__metric': 'euclidean', 'model__n_neighbors': 17}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237D26936D0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000237CE9DA6D0>)])),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=17))])

## Tunability 

In [12]:
df_knn['mean_f1'] = df_knn[['credit_f1', 'college_f1', 'diabetes_f1', 'penguins_f1']].mean(axis=1)
df_knn

,model__n_neighbors,model__metric,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
0,1,euclidean,0.780474,0.833291,0.574474,0.985714,0.793488
1,2,euclidean,0.744687,0.803480,0.481986,0.985714,0.753967
2,3,euclidean,0.839961,0.845150,0.619668,0.989474,0.823563
3,4,euclidean,0.810188,0.809347,0.574451,0.989474,0.795865
4,5,euclidean,0.829706,0.827496,0.603450,0.989474,0.812531
...,...,...,...,...,...,...,...
115,26,minkowski,0.835261,0.781369,0.626080,0.985714,0.807106
116,27,minkowski,0.841074,0.778332,0.636465,0.989474,0.811336
117,28,minkowski,0.826965,0.777235,0.615963,0.989474,0.802409
118,29,minkowski,0.829547,0.788058,0.618305,0.989474,0.806346


In [13]:
# sigma star
sigma_star_row = df_knn[df_knn['mean_f1'] == max(df_knn['mean_f1'])]
sigma_star_row

,model__n_neighbors,model__metric,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
10,11,euclidean,0.845129,0.819695,0.644186,0.993103,0.825528
100,11,minkowski,0.845129,0.819695,0.644186,0.993103,0.825528


### (3.3) Measuring Overall Tunability of a ML Algorithm

In [14]:
d_credit = -sigma_star_row['credit_f1'].iloc[0] + best_f1_credit
d_college = -sigma_star_row['college_f1'].iloc[0] + best_f1_college
d_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_f1_diabetes
d_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_f1_penguins

d = pd.Series([d_credit, d_college, d_diabetes, d_penguins])
print(d)
print(f"Mean: {d.mean()}")


0    0.000187
1    0.025455
2    0.000000
3    0.006897
dtype: float64
Mean: 0.008134652928707964


### Saving results

In [15]:
df_knn.to_csv("../../data/RSmodels/knn_results.csv", index=False)

# BayesSearchCV

In [31]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

n_iter_BS = pd.read_csv('../../data/vars.csv')['n_iter_BS'].iloc[0]
BS_iterations = pd.read_csv('../../data/vars.csv')['BS_iterations'].iloc[0]

knn_params = {
    'model__n_neighbors': Integer(1, 31),
    #'model__weights': Categorical(['uniform', 'distance']),
    #'model__algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
    'model__metric': Categorical(['euclidean', 'manhattan', 'chebyshev', 'minkowski'])
}

## Credit dataset

In [32]:
X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]


bayes_search_credit_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_credit_model.fit(X, y)

best_BS_f1_credit = bayes_search_credit_model.best_score_
credit_best_estimator = bayes_search_credit_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_credit}")

credit_best_estimator

Best f1 score: 0.8451285323122008


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C775B8D0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C77A3BD0>)])),
                ('model', KNeighborsClassifier(n_neighbors=11))])

## College dataset

In [33]:
X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]


bayes_search_college_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_college_model.fit(X, y)

best_BS_f1_college = bayes_search_college_model.best_score_
college_best_estimator = bayes_search_college_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_college}")

college_best_estimator

Best f1 score: 0.8451496931440531


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C79BE410>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C79BF810>)])),
                ('model', KNeighborsClassifier(n_neighbors=3))])

## Diabetes dataset

In [34]:
X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]


bayes_search_diabetes_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_diabetes_model.fit(X, y)

best_BS_f1_diabetes = bayes_search_diabetes_model.best_score_
diabetes_best_estimator = bayes_search_diabetes_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_diabetes}")
diabetes_best_estimator

Best f1 score: 0.6441857768953202


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C78DCE90>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C9B624D0>)])),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=11))])

## Penguins dataset

In [35]:
X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]


bayes_search_penguins_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_penguins_model.fit(X, y)

best_BS_f1_penguins = bayes_search_penguins_model.best_score_
penguins_best_estimator = bayes_search_penguins_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_penguins}")

penguins_best_estimator

Best f1 score: 1.0


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C9AD7F10>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000103C9BB3A50>)])),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=17))])

## Tunability

### (3.3) Measuring Overall Tunability of a ML Algorithm

In [36]:
d_BS_credit = -sigma_star_row['credit_f1'].iloc[0] + best_BS_f1_credit
d_BS_college = -sigma_star_row['college_f1'].iloc[0] + best_BS_f1_college
d_BS_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_BS_f1_diabetes
d_BS_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_BS_f1_penguins

d_BS = pd.Series([d_BS_credit, d_BS_college, d_BS_diabetes, d_BS_penguins])
print(d_BS)
print(f"Mean: {d_BS.mean()}")

0    0.000000
1    0.025455
2    0.000000
3    0.006897
dtype: float64
Mean: 0.008087843523484267


### Models saving

In [37]:
from joblib import dump

dump(bayes_search_college_model, '../../data/BSmodels/KNeighbors/bayes_search_college_model.joblib')
dump(bayes_search_credit_model, '../../data/BSmodels/KNeighbors/bayes_search_credit_model.joblib')
dump(bayes_search_diabetes_model, '../../data/BSmodels/KNeighbors/bayes_search_diabetes_model.joblib')
dump(bayes_search_penguins_model, '../../data/BSmodels/KNeighbors/bayes_search_penguins_model.joblib')

['../../data/BSmodels/KNeighbors/bayes_search_penguins_model.joblib']